In [2]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import glob

In [3]:
# Cоздаем листы тегов - общий, без параграфов и с параграфами
сommon_lst = ['p','title', 'stanza', 'epigraph', 'poem', 'author', 'cite','annotation', 'subtitle', 'publisher', 'keywords', 'book-title']
lst_without_paragraph = ['title', 'epigraph', 'poem', 'author', 'cite']
lst =['annotation', 'subtitle', 'publisher', 'keywords', 'book-title']

In [4]:
# Определяем функцию для приготовления общего словаря
def annotation_analysis_all(tag, dctbook_all):
    annotations = parsed_book.find_all(tag)
    for annotation in annotations:
        text = annotation.text.split('\n')
        for paragraph in text:
            if paragraph != '':
                concat = paragraph
                dctbook_all[concat] = tag

In [5]:
# Определяем функцию для приготовления словаря всех тегов, кроме р
def annotation_analysis_no_p(tag, dctbook_no_p, par_index):
    annotations = parsed_book.find_all(tag)
    for annotation in annotations:
        text = annotation.text.split('\n')
        if par_index == 1:
            for paragraph in text:
                if paragraph != '':
                    concat = paragraph
                    dctbook_no_p[concat] = tag
        elif par_index !=1:
            paragraph_new = ' '.join([str(paragraph)for paragraph in text])
            if paragraph_new != '':
                concat = paragraph_new
                dctbook_no_p[concat]=tag

In [6]:
filesMask="Books/*.fb2"    # маска для файлов, которые обрабатываем
resultFiles = glob.glob(filesMask) # Эта функция выдает список имен файлов по заданной маске в текущем каталоге

In [7]:
resultFiles

['Books/Петровский (Федот Бега, Владимир Александров) - 1963.fb2',
 'Books/Полководцы гражданской войны (Сборник) - 1960.fb2',
 'Books/Пикассо (Анри Жидель) - 2007.fb2',
 'Books/Петр Алексеев (Островер Л.) - 1964.fb2',
 'Books/Петр Столыпин. Крестный путь реформатора (Дмитрий Табачник, Виктор Воронин) - 2012.fb2',
 'Books/Прасковья Ангелина (Аркадий Славутский) - 1960.fb2',
 'Books/Плешаков (Владимир Бирюков, Николай Бодрихин) - 2012.fb2',
 'Books/Петр Смородин (Владимир Архангельский) - 1974.fb2',
 'Books/Праймашина.fb2',
 'Books/Петр III (Александр Сергеевич Мыльников) - 2002.fb2',
 'Books/19. Дураки умирают первыми.fb2',
 'Books/Петр Первый (Николай Павленко) - 1976.fb2',
 'Books/Поль Робсон (Владимир Зимянин) - 1985.fb2',
 'Books/Прянишников (Олег Писаржевский) - 1963.fb2',
 'Books/Президенты США (Георгий Чернявский, Лариса Дубова) - 2019.fb2',
 'Books/Песталоцци. Воспитатель человечества (Андрей Максимов) - 2023.fb2',
 'Books/Павленков (Владимир Десятерик) - 2006.fb2',
 'Books/Пат

In [8]:
# Импортируем файлы и на их основе делаем словарь, ключами которого являются тексты, а значениями - теги.

dctbook_main = {} # Наш словарь
dctbook_all = {} # Вспомогательный словарь
dctbook_no_p = {} # Вспомогательный словарь

num_file=1
num = 1
# 1. Генерируем словарь тегов из общего списка.
for filename in resultFiles:      # Цикл по именам файлов
    print("Processing file: ",filename)

    dctbook_common = {}
    
    try:
        with open(filename, encoding="utf-8") as file:
            src = file.read()
            parsed_book = BeautifulSoup(src, 'xml')
            dctbook_all = {}
            for i in range(len(сommon_lst)):
                annotation_analysis_all(сommon_lst[i], dctbook_all)

            

        # 2. Генерируем словарь тегов из списков без тега <p>
            for i in range(len(lst_without_paragraph)):
                annotation_analysis_no_p(lst_without_paragraph[i], dctbook_no_p, 0)
            for i in range(len(lst)):
                annotation_analysis_no_p(lst[i], dctbook_no_p, 1)
        
        # 3. Генерируем словарь примечаний
            dct_notes = {}
            annotations = parsed_book.find_all('body',{"name":"notes"})
            notes = str(annotations).split('\n')
            for string in notes:
                if '<p>' in string:
                    string_modified = string[3:]
                    end = string_modified.rfind('<')
                    string = string_modified[:end]
                    if len(string)> 4: 
                        dct_notes[string] = 'note'
        

        # 4. На основе полученных словарей собираем общий, где не будет перекрытия по тегам.
            for key in dctbook_all.keys():
                if dctbook_all[key] == 'p':
                    dctbook_common[key] = dctbook_all[key]
            for key in dctbook_no_p.keys():
                dctbook_common[key]=dctbook_no_p[key]
            for key in dct_notes.keys():
                try:
                    if dctbook_common[key] == 'p':
                        dctbook_common[key] = 'note'
                except Exception:
                    print(f'Какая-то ошибка с парсингом абзацев {num} в файле {filename}')
                    num+=1
                    dctbook_common = {}
    
            for key in dctbook_common.keys():
                dctbook_main[key] = dctbook_common[key]

    except Exception:
        print(f'Файл {filename} загрузился с ошибкой и будет исключен из парсинга. С ошибкой загрузилось {num_file} файлов')
        num_file+=1
        continue

Processing file:  Books/Петровский (Федот Бега, Владимир Александров) - 1963.fb2
Файл Books/Петровский (Федот Бега, Владимир Александров) - 1963.fb2 загрузился с ошибкой и будет исключен из парсинга. С ошибкой загрузилось 1 файлов
Processing file:  Books/Полководцы гражданской войны (Сборник) - 1960.fb2
Файл Books/Полководцы гражданской войны (Сборник) - 1960.fb2 загрузился с ошибкой и будет исключен из парсинга. С ошибкой загрузилось 2 файлов
Processing file:  Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 1 в файле Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 2 в файле Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 3 в файле Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 4 в файле Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 5 в файле Books/Пикассо (Анри Жидель) - 2007.fb2
Какая-то ошибка с парсингом абзацев 6 в файле Books/Пикассо (Анри Жиде

In [ ]:
# 5. Демонстрация общего словаря.
for key in dctbook_main:
    print(key, dctbook_main[key])

In [9]:
df = pd.DataFrame.from_dict(dctbook_main, orient='index')

In [10]:
df.to_csv('book_fb2_dataset.csv', sep='\t')